## Coding assignment: Graph Neural Networks (GNN)

Graph structures are ubiquitous in various domains, from social networks to molecular interactions. Understanding these complex relationships requires advanced analytical tools, and Graph Neural Networks (GNNs) provide a powerful framework for extracting meaningful insights from graph-structured data.

In this assignment, you will:

- Gain a foundational understanding of GNNs and their underlying principles,
- Explore their applications in graph analysis, and
- Implement GNNs using state-of-the-art deep learning frameworks.
-
By the end of this assignment, you will have acquired both theoretical knowledge and hands-on experience in applying GNNs to real-world graph data.

## Environment Setup
For a seamless execution of this notebook, ensure your Python environment is properly set up. Here's what you'll need:

Python Version: We recommend using Python 3.8 or higher.

Required Packages: Install the following libraries to delve into GNNs:

```
torch
torch_geometric
torch_scatter
torch_sparse
torchmetrics
networkx
numpy
jupyter
```

In [1]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.5.1+cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 40.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.6 MB/s eta 0:00:00


In this assignment, we will utilize the **CLUSTER** dataset from the GNNBenchmarkDataset, as introduced in the paper Benchmarking Graph Neural Networks. This dataset is a subset of the Stochastic Block Model (SBM) datasets, which focus on node-level graph pattern recognition tasks, originally explored by Scarselli et al. (2009). Specifically, it addresses the following tasks:
- Graph Pattern Recognition (PATTERN)
- Semi-supervised Graph Clustering (CLUSTER)
The Stochastic Block Model (SBM), as described by Abbe (2017), serves as the foundation of these datasets. SBM is widely used for modeling community structures in social networks, where intra- and inter-community connections are probabilistically controlled. In particular:

- Two nodes within the same community are connected with probability p.
- Nodes belonging to different communities are connected with probability q, which acts as a noise parameter.

The CLUSTER dataset has the following properties:

- Each node is represented by a 7-dimensional feature vector.
- The dataset contains 6 distinct node classes.
- The primary learning objective is multi-class classification at the node level.

This dataset provides a challenging yet insightful benchmark for evaluating the performance of Graph Neural Networks in community detection and clustering tasks. We will start by loading the dataset and take a look at the graphs in the dataset.

In [2]:
import torch
import torch.nn as nn
from torch_geometric.datasets import GNNBenchmarkDataset
from torch_geometric.loader import DataLoader
import numpy as np
import matplotlib.pyplot as plt

# Load the Cluster dataset
data_root = './data'
dataset_train = GNNBenchmarkDataset(root=data_root, name='CLUSTER', split='train')
dataset_val = GNNBenchmarkDataset(root=data_root, name='CLUSTER', split='val')

# Print dataset statistics
print(f'Training dataset size: {len(dataset_train)}')
print(f'Validation dataset size: {len(dataset_val)}')

# Print a sample graph's details
def print_graph_info(graph, index):
    print(f'Graph {index}:')
    print(f'  - Number of nodes: {graph.num_nodes}')
    print(f'  - Number of edges: {graph.num_edges}')
    print(f'  - Node features shape: {graph.x.shape}')
    print(f'  - Edge index shape: {graph.edge_index.shape}')
    print(f'  - Labels shape: {graph.y.shape}')
    print('-' * 40)

# Display information about the first few graphs
for i in range(min(1, len(dataset_train))):
    print_graph_info(dataset_train[i], i)


Extracting data/CLUSTER/raw/CLUSTER_v2.zip
Processing...
Done!


Training dataset size: 10000
Validation dataset size: 1000
Graph 0:
  - Number of nodes: 117
  - Number of edges: 4104
  - Node features shape: torch.Size([117, 7])
  - Edge index shape: torch.Size([2, 4104])
  - Labels shape: torch.Size([117])
----------------------------------------


Then we use torch_geometric's data loader class to wrap the dataset to batches:

In [3]:
# Create DataLoaders
batch_size = 32
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
dataloader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=False)

# Display batched graph information
for batch in dataloader_train:
    print('Batched graph:')
    print(f'  - Batch size: {batch_size}')
    print(f'  - Total nodes in batch: {batch.x.shape[0]}')
    print(f'  - Total edges in batch: {batch.edge_index.shape[1]}')
    print(f'  - Labels shape: {batch.y.shape}')
    break  # Only show one batch

Batched graph:
  - Batch size: 32
  - Total nodes in batch: 3733
  - Total edges in batch: 138014
  - Labels shape: torch.Size([3733])


## Task 1: MLP for node classification

To start with, we will build an MLP model as a baseline. The MLP should directly take the node features as input and output the predictions for each class. In this task, you need to complete the MLP model and the training function.

In [ ]:
class MLPNodeClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        #####
        super(MLPNodeClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        #####

    def forward(self, x):
        #####
        x = torch.nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x
        #####

The training and evaluation functions for the MLP model:

In [ ]:
# Evaluation function
def evaluate(model, dataloader, device='cuda:0'):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in dataloader:
            #####
            inputs = batch.x
            labels = batch.y
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs, dim=1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            #####
    # return the accuracy
    return correct / total

# Training function
def train(model, dataloader_train, dataloader_val, epochs=50, lr=0.001, patience=5, device='cuda:0'):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    best_val_acc = 0
    patience_counter = 0

    model.to(device)
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct = 0 # number of correct predicted nodes
        total = 0 # number of nodes

        for batch in dataloader_train:
            #####
            inputs = batch.x
            labels = batch.y  # Assuming dataloader returns (inputs, labels)
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs, dim=1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            #####

        train_acc = correct / total
        val_acc = evaluate(model, dataloader_val, device)
        print(f'Epoch {epoch+1}: Loss={total_loss:.4f}, Train Acc={train_acc:.4f}, Val Acc={val_acc:.4f}')

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
            best_ckpt = model.state_dict()
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print('Early stopping triggered')
            break
    return best_ckpt

We will train the MLP for the node classification task. There are several hyperparameters that might need your attention, including the batch size, hidden dimension of the MLP, number of epochs, learning rate, and early stop patience.

In [6]:
# Initialize model
input_dim = dataset_train.num_node_features
hidden_dim = 1280
output_dim = dataset_train.num_classes

model = MLPNodeClassifier(input_dim, hidden_dim, output_dim)
print(model)

# Train the model
best_ckpt = train(model, dataloader_train, dataloader_val, epochs=10, lr=1e-4, patience=3)

MLPNodeClassifier(
  (fc1): Linear(in_features=7, out_features=1280, bias=True)
  (fc2): Linear(in_features=1280, out_features=6, bias=True)
)
Epoch 1: Loss=546.3391, Train Acc=0.2066, Val Acc=0.2133
Epoch 2: Loss=534.6440, Train Acc=0.2086, Val Acc=0.2105
Epoch 3: Loss=532.9793, Train Acc=0.2093, Val Acc=0.2105
Epoch 4: Loss=532.5838, Train Acc=0.2101, Val Acc=0.2105
Early stopping triggered


After the training, we will make predictions on the test set and save the prediction results. The saved predictions will be used for grading.

In [ ]:
dataset_test = GNNBenchmarkDataset(root=data_root, name='CLUSTER', split='test')
dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

# Test set predictions and save results
def predict(model, dataloader, filename='predictions.txt', device='cuda:0'):
    model.eval()
    predictions = [] # a list of predicted labels
    with torch.no_grad():
        for batch in dataloader:
            #####
            inputs = batch.x
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, dim=1)
            predictions.extend(predicted.cpu().numpy())
            #####

    return predictions

model.load_state_dict(best_ckpt)
predictions = predict(model, dataloader_test)
np.savetxt('predictions_mlp_cluster.txt', predictions, fmt='%d')

## Task 2: GCN for node classification

Next we will leverage the graph convolutional layers to construct a GNN model for the node classification task. You can check pyg's [documentation](https://pytorch-geometric.readthedocs.io/en/latest/) to learn the usage of the `GCNConv` module and use it to build a GNN model. In the following, you need to complete the `GNNNodeClassifier` class as well as the training function.

In [ ]:
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
# Define a GNN model using GCNConv
class GNNNodeClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_gcn_layers=10):
        #####
        super(GNNNodeClassifier, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)
        #####

    def forward(self, x, edge_index):
        #####
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x
        #####

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Training function
def train(model, dataloader_train, dataloader_val, epochs=50, lr=0.001, patience=5, device='cuda:0'):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=5, verbose=True)
    best_val_acc = 0
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for batch in dataloader_train:
            #####
            inputs = batch.x
            labels = batch.y
            edge_index = batch.edge_index
            inputs, labels, edge_index = inputs.to(device), labels.to(device), edge_index.to(device)
            optimizer.zero_grad()
            outputs = model(inputs,edge_index)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            _, predicted = torch.max(outputs, dim=1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            #####

        train_acc = correct / total
        val_acc = evaluate(model, dataloader_val, device)
        scheduler.step(val_acc)
        print(f'Epoch {epoch+1}: Loss={total_loss:.4f}, Train Acc={train_acc:.4f}, Val Acc={val_acc:.4f}')

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print('Early stopping triggered')
            break

# Evaluation function
def evaluate(model, dataloader, device='cuda:0'):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in dataloader:
            #####
            inputs = batch.x
            labels = batch.y
            edge_index = batch.edge_index
            inputs, labels, edge_index = inputs.to(device), labels.to(device), edge_index.to(device)

            outputs = model(inputs,edge_index)
            _, predicted = torch.max(outputs, dim=1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            #####
    return correct / total

Then we can train the GCN model for the node classification. You should carefully choose the hyperparameters: batch size, hidden dimension, number of GCN layers, number of epochs, learning rate, and early stop patience.

In [11]:
# Initialize model
input_dim = dataset_train.num_node_features
hidden_dim = 512
output_dim = dataset_train.num_classes

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = GNNNodeClassifier(input_dim, hidden_dim, output_dim, num_gcn_layers=5)
print(model)
print(f'Number of trainable params: {sum(p.numel() for p in model.parameters() if p.requires_grad)}')
model.to(device)

# Train the model
train(model, dataloader_train, dataloader_val, epochs=30, lr=1e-3, patience=5, device=device)

GNNNodeClassifier(
  (conv1): GCNConv(7, 512)
  (conv2): GCNConv(512, 6)
)
Number of trainable params: 7174
Epoch 1: Loss=559.0032, Train Acc=0.2157, Val Acc=0.3102
Epoch 2: Loss=548.5375, Train Acc=0.3352, Val Acc=0.3697
Epoch 3: Loss=526.3581, Train Acc=0.4155, Val Acc=0.4252
Epoch 4: Loss=501.2557, Train Acc=0.4417, Val Acc=0.4523
Epoch 5: Loss=481.8448, Train Acc=0.4533, Val Acc=0.4517
Epoch 6: Loss=468.5315, Train Acc=0.4580, Val Acc=0.4586
Epoch 7: Loss=459.4337, Train Acc=0.4599, Val Acc=0.4583
Epoch 8: Loss=453.3189, Train Acc=0.4609, Val Acc=0.4606
Epoch 9: Loss=448.8634, Train Acc=0.4626, Val Acc=0.4613
Epoch 10: Loss=445.7194, Train Acc=0.4631, Val Acc=0.4603
Epoch 11: Loss=443.5800, Train Acc=0.4635, Val Acc=0.4617
Epoch 12: Loss=441.9017, Train Acc=0.4641, Val Acc=0.4637
Epoch 13: Loss=440.7250, Train Acc=0.4644, Val Acc=0.4649
Epoch 14: Loss=439.8229, Train Acc=0.4648, Val Acc=0.4637
Epoch 15: Loss=439.0982, Train Acc=0.4655, Val Acc=0.4640
Epoch 16: Loss=438.5346, Train 

After training, we can make predictions on the test set and save the results. The results will be used for the grading of the task.

In [ ]:
# Test set predictions and save results
def predict(model, dataloader, filename='gnn_predictions.txt', device='cuda:0'):
    model.eval()
    predictions = []
    with torch.no_grad():
        for batch in dataloader:
            #####
            inputs = batch.x
            labels = batch.y
            edge_index = batch.edge_index
            inputs, labels, edge_index = inputs.to(device), labels.to(device), edge_index.to(device)

            outputs = model(inputs,edge_index)
            _, predicted = torch.max(outputs, dim=1)
            predictions.extend(predicted.cpu().numpy())
            #####

    return predictions


predictions = predict(model, dataloader_test)
np.savetxt('predictions_gcn_cluster.txt', predictions, fmt='%d')

## Graph classification task

In this section, we explore graph classification using Graph Neural Networks (GNNs). Unlike node classification, which focuses on predicting labels for individual nodes within a graph, graph classification aims to assign labels to entire graphs based on their structural and feature-based attributes. The primary challenge lies in effectively embedding entire graphs into a feature space where they become linearly separable for classification tasks.

One notable application of graph classification is the representation of image data as graphs, an approach demonstrated in super-pixel datasets. These datasets provide a novel way to transform traditional image classification tasks into graph learning problems. Prominent image datasets such as MNIST and CIFAR10 have been adapted into graph structures using this methodology. The motivation for utilizing these datasets is twofold:

- Benchmarking and Sanity-Checking – These datasets serve as standard benchmarks for evaluating the performance of GNN architectures. Most GNN models are expected to achieve near-perfect accuracy on MNIST and competitive performance on CIFAR10.
- Extending Image-Based Learning to Graphs – Super-pixel representations offer valuable insights into how conventional image datasets can be leveraged for graph-based learning and analysis.

### CIFAR10 Super-Pixel Dataset
In this assignment, we will work with the CIFAR10 super-pixel dataset for a graph classification task. The CIFAR10 images are transformed into graphs using super-pixel segmentation, where each super-pixel represents a small, homogeneous region of the image. This transformation is performed using the Simple Linear Iterative Clustering (SLIC) algorithm, introduced by Achanta et al. (2012).

By leveraging super-pixel representations, we can analyze the effectiveness of GNNs in graph classification while drawing connections between traditional computer vision tasks and graph-based learning techniques.

In [31]:
from torch_geometric.nn import GCNConv, GATConv, global_mean_pool, global_max_pool, GraphNorm

# Load the CIFAR10 dataset
data_root = './data/GNNBenchmark'
dataset_train = GNNBenchmarkDataset(root=data_root, name='CIFAR10', split='train')
dataset_val = GNNBenchmarkDataset(root=data_root, name='CIFAR10', split='val')

print(f'Training dataset size: {len(dataset_train)}')
print(f'Validation dataset size: {len(dataset_val)}')
print_graph_info(dataset_train[0], 0)

Training dataset size: 45000
Validation dataset size: 5000
Graph 0:
  - Number of nodes: 110
  - Number of edges: 880
  - Node features shape: torch.Size([110, 3])
  - Edge index shape: torch.Size([2, 880])
  - Labels shape: torch.Size([1])
----------------------------------------


## Task 3: GCN for graph classification

In this task, you need to build a GNN model using the `GCNConv` module for the graph classification task. Note that in order to do graph classification, you need to get the graph embedding by pooling the node embeddings. You can refer to pyg's [documentation](https://pytorch-geometric.readthedocs.io/en/latest/) for different pooling functions (e.g., mean pooling, max pooling, sum pooling).

In [ ]:
# Define a GCN model for graph classification
class GCNGraphClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_gcn_layers=2):
        #####
        super(GCNGraphClassifier, self).__init__()

        #Define GCN layers
        self.convs = nn.ModuleList()
        self.norms = nn.ModuleList()
        self.convs.append(GCNConv(input_dim, hidden_dim))  #First layer
        self.norms.append(GraphNorm(hidden_dim))

        for _ in range(num_gcn_layers - 2):
            self.convs.append(GCNConv(hidden_dim, hidden_dim))  #Hidden layers
            self.norms.append(GraphNorm(hidden_dim))
        self.convs.append(GCNConv(hidden_dim, output_dim))  #Final GCN layer

        #Global pooling layer to obtain graph-level representation
        self.pool = global_mean_pool

        #Fully connected layer for final classification
        self.fc = nn.Linear(output_dim, 10)
        #####

    def forward(self, x, edge_index, batch):
        #####
        for i in range(len(self.convs) - 1):  # Apply GraphNorm to hidden layers only
            x = self.convs[i](x, edge_index)
            x = self.norms[i](x)  # Correct usage of GraphNorm
            x = F.relu(x)

        x = self.convs[-1](x, edge_index)
        # Aggregate node embeddings into a graph representation
        x = self.pool(x, batch)

        # Fully connected classification layer
        return self.fc(x)
        #####

In [ ]:
# Training function
def train(model, dataloader_train, dataloader_val, epochs=50, lr=0.001, patience=5, device='cuda:0'):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    best_val_acc = 0
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for batch in dataloader_train:
            #####
            batch = batch.to(device)  # Move batch to device (GPU or CPU)

            optimizer.zero_grad()
            outputs = model(batch.x, batch.edge_index, batch.batch)  # Forward pass
            loss = criterion(outputs, batch.y)  # Compute loss
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs, dim=1)  # Get predicted class indices
            correct += (predicted == batch.y).sum().item()
            total += batch.y.size(0)
            #####

        train_acc = correct / total
        val_acc = evaluate(model, dataloader_val, device)
        print(f'Epoch {epoch+1}: Loss={total_loss:.4f}, Train Acc={train_acc:.4f}, Val Acc={val_acc:.4f}')

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print('Early stopping triggered')
            break

# Evaluation function
def evaluate(model, dataloader, device='cuda:0'):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in dataloader:
            #####
            batch = batch.to(device)  # Move batch to device
            outputs = model(batch.x, batch.edge_index, batch.batch)  # Forward pass
            _, predicted = torch.max(outputs, dim=1)  # Get predicted class indices
            correct += (predicted == batch.y).sum().item()
            total += batch.y.size(0)
            #####
    return correct / total

Train the GCN model for graph classification:

In [36]:
# Create DataLoaders
batch_size = 32
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
dataloader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=False)

# Initialize models
input_dim = dataset_train.num_node_features
hidden_dim = 128
output_dim = dataset_train.num_classes

gcn_model = GCNGraphClassifier(input_dim, hidden_dim, output_dim, num_gcn_layers=8)
gcn_model.to(device)
print(gcn_model)
print(f'Number of trainable params: {sum(p.numel() for p in gcn_model.parameters() if p.requires_grad)}')

# Train the GCN model
train(gcn_model, dataloader_train, dataloader_val, epochs=50, lr=1e-3, patience=5, device=device)

GCNGraphClassifier(
  (convs): ModuleList(
    (0): GCNConv(3, 128)
    (1-6): 6 x GCNConv(128, 128)
    (7): GCNConv(128, 10)
  )
  (norms): ModuleList(
    (0-6): 7 x GraphNorm(128)
  )
  (fc): Linear(in_features=10, out_features=10, bias=True)
)
Number of trainable params: 103672
Epoch 1: Loss=2817.7209, Train Acc=0.2635, Val Acc=0.2824
Epoch 2: Loss=2740.4002, Train Acc=0.2847, Val Acc=0.2880
Epoch 3: Loss=2713.3685, Train Acc=0.2968, Val Acc=0.2920
Epoch 4: Loss=2693.4344, Train Acc=0.3022, Val Acc=0.2990
Epoch 5: Loss=2677.7845, Train Acc=0.3062, Val Acc=0.3122
Epoch 6: Loss=2656.2311, Train Acc=0.3128, Val Acc=0.3106
Epoch 7: Loss=2642.4032, Train Acc=0.3160, Val Acc=0.3100
Epoch 8: Loss=2628.8232, Train Acc=0.3208, Val Acc=0.3218
Epoch 9: Loss=2619.3878, Train Acc=0.3225, Val Acc=0.3240
Epoch 10: Loss=2603.8775, Train Acc=0.3282, Val Acc=0.3244
Epoch 11: Loss=2595.3688, Train Acc=0.3306, Val Acc=0.3216
Epoch 12: Loss=2591.8770, Train Acc=0.3303, Val Acc=0.3320
Epoch 13: Loss=25

After training, you can make predictions using the GCN model. Save the prediction results in a `.txt` file, where each line contains one prediction for one test data point.

In [ ]:
def predict(model, dataloader, device='cuda:0'):
    model.eval()
    predictions = []
    with torch.no_grad():
        for batch in dataloader:
            #####
            batch = batch.to(device)  # Move batch to device
            outputs = model(batch.x, batch.edge_index, batch.batch)  # Forward pass
            _, predicted = torch.max(outputs, dim=1)  # Get predicted class indices
            predictions.extend(predicted.cpu().numpy())
            #####

    return predictions

dataset_test = GNNBenchmarkDataset(root=data_root, name='CIFAR10', split='test')
dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

predictions_gcn = predict(gcn_model, dataloader_test)
np.savetxt('predictions_gcn_cifar10.txt', predictions_gcn, fmt='%d')

## Grading

All the tasks will be graded by the accuracy on the test set.

### Task 1 (5 points)
- Accuracy >= 0.2: 5 points
- Accuracy < 0.2: 0 points

### Task 2 (10 points)
- Accuracy >= 0.32: 10 points
- Accuracy < 0.32: 0 points

### Task 3 (10 points)
- Accuracy >= 0.4: 10 points
- Accuracy < 0.4: 0 points


## Submission

After completing all the tasks, you should submit the following four files to Gradescope:

- `hw4_gnn.ipynb`: The notebook with all tasks completed.
- `predictions_mlp_cluster.txt`: prediction results of the MLP model on CLUSTER dataset.
- `predictions_gcn_cluster.txt`: prediction results of the GCN model on CLUSTER dataset.
- `predictions_gcn_cifar10.txt`: prediction results of the GCN model on cifar10 dataset.

Note that you need to submit the files individually, **DO NOT** submit a zip file.